# Ratings

This notebook is meant to add new features to our listings: the ratings

In [1]:
import time   
from random import randint
from bs4 import BeautifulSoup
import re
import urllib
import requests
import pandas as pd
import numpy as np

def get_result(room_):
    hdr = {'User-Agent': 
           'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.11 (KHTML, like Gecko) Chrome/23.0.1271.64 Safari/537.11'}
    room=str(room_)
    url = "https://www.airbnb.com/rooms/"
    search= '?guests=1&adults=1'
    url_=url+room+search
    time.sleep(randint(0,1))
    response = requests.get(url_, hdr, timeout=20)
    result_page = BeautifulSoup(response.content,'lxml')
    
    return result_page

def get_avail(room_):
    result_page = get_result(room_)
    meta_=result_page.find_all(id="reviews") #If empty, listing is not available
    if meta_==[]:
        available_=False
    else:
        available_=True

    return available_

In [2]:
def ratings(room_): 
    result_page = get_result(room_)
    available_=get_avail(room_)
        
    try:
        
        if available_==False:
            rating_df=pd.DataFrame([[np.NAN, np.NAN, np.NAN, np.NAN, np.NAN, np.NAN, np.NAN]],index = [room_], 
                    columns = ['Stars','Accuracy','Communication','Cleanliness','Location','Check-In','Value'])
    
        else:
            abc_=result_page.find_all(type="application/json")[2].get_text()

            # AirBNB has 6 main categories in the reviews:
            # Accuracy, Communication, Cleanliness, Location, Check-in & Value

            acc_=(abc_[abc_.rfind('"label":"Accuracy"')-4:abc_.rfind('"label":"Accuracy"')]).replace(':',"").replace(',',
                                                                    "").replace('"',"").replace('}',"").replace('-',"")
            comm_=(abc_[abc_.rfind('"label":"Communication"')-4:abc_.rfind('"label":"Communication"')]).replace(':',
                                                "").replace(',',"").replace('"',"").replace('}',"").replace('-',"")
            cle_=(abc_[abc_.rfind('"label":"Cleanliness"')-4:abc_.rfind('"label":"Cleanliness"')]).replace(':',
                                            "").replace(',',"").replace('"',"").replace('}',"").replace('-',"")
            loc_=(abc_[abc_.rfind('"label":"Location"')-4:abc_.rfind('"label":"Location"')]).replace(':',
                                    "").replace(',',"").replace('"',"").replace('}',"").replace('-',"")
            che_=(abc_[abc_.rfind('"label":"Check-in"')-4:abc_.rfind('"label":"Check-in"')]).replace(':',
                                    "").replace(',',"").replace('"',"").replace('}',"").replace('-',"")
            val_=(abc_[abc_.rfind('"label":"Value"')-4:abc_.rfind('"label":"Value"')]).replace(':',
                            "").replace(',',"").replace('"',"").replace('}',"").replace('-',"")
            star_=(abc_[abc_.find('"star_rating"')+14:abc_.find('"star_rating"')+19]).replace(':',
                                 "").replace(',',"").replace('"',"").replace('t','').replace('i','').replace('ralU',"")

            # Ratings
            rat_=[star_,acc_,comm_,cle_,loc_,che_,val_]
            arr_=np.array(rat_)
            arr_=[arr_]
            lab_=['Stars','Accuracy','Communication','Cleanliness','Location','Check-In','Value']

            rating_df=pd.DataFrame(arr_,index=[room_],columns=lab_)

        return rating_df
        
    except:
        
        print("Didn't work",'Room:',room_)
        return pd.DataFrame([[np.NAN, np.NAN, np.NAN, np.NAN, np.NAN, np.NAN, np.NAN]],index = [room_], columns = ['Stars',
                                    'Accuracy','Communication','Cleanliness','Location','Check-In','Value'])
    

In [34]:
filename = 'listings_neibourhood.csv'
data_borough = pd.read_csv(filename)
del data_borough['Unnamed: 0']
data_borough.head()

,id,price,minimum_nights,number_of_reviews,reviews_per_month,availability_365,room_type_Entire home/apt,room_type_Private room,room_type_Shared room,neighbourhood_Harlem,neighbourhood_Morningside Heights,neighbourhood_Upper West Side
0,3647,150,3,0,0.00,365,0,1,0,1,0,0
1,5203,79,2,118,1.03,0,0,1,0,0,0,1
2,5295,135,5,43,0.36,53,1,0,0,0,0,1
3,6021,85,2,100,0.84,365,0,1,0,0,0,1
4,9668,50,3,230,2.02,348,0,1,0,1,0,0


In [9]:
ids = data_borough['id']
len(ids)

5296

In [16]:
ids.head()

0    3647
1    5203
2    5295
3    6021
4    9668
Name: id, dtype: int64

Data frame is generated in small batch in order to mitigate mistakes due to internet connections, loading errors, etc. There is a timeout between each call to avoid blockage from AirBNB

In [5]:
#Generating Data Frame 
rating_ = ratings(ids.iloc[0])

In [6]:
#Generating 1st 1000 entries
for i in range(1,1000):
    rating_ = rating_.append(ratings(ids.iloc[i]))
    if i%50 == 0:
        print('step',i)
print('done')

step 50
step 100
step 150
step 200
step 250
step 300
step 350
step 400
step 450
step 500
step 550
step 600
step 650
Didn't work Room: 3818298
step 700
Didn't work Room: 4121110
step 750
step 800
step 850
step 900
step 950
done


In [12]:
#Generating next entries
for i in range(1000,1500):
    rating_ = rating_.append(ratings(ids.iloc[i]))
    if i%50 == 0:
        print('step',i)
print('done')

step 1000
step 1050
step 1100
Didn't work Room: 6471117
step 1150
step 1200
Didn't work Room: 7034609
Didn't work Room: 7279583
step 1250
step 1300
step 1350
step 1400
step 1450
done


In [29]:
#Generating next entries
for i in range(1500,2000):
    rating_ = rating_.append(ratings(ids.iloc[i]))
    if i%50 == 0:
        print('step',i)
print('done')

step 1500
Didn't work Room: 9022118
step 1550
Didn't work Room: 9249597
Didn't work Room: 9296089
Didn't work Room: 9405466
step 1600
step 1650
Didn't work Room: 9839017
step 1700
step 1750
step 1800
Didn't work Room: 10367220
Didn't work Room: 10704981
step 1850
step 1900
step 1950
done


In [81]:
#Generating next entries
for i in range(2000,2500):
    rating_ = rating_.append(ratings(ids.iloc[i]))
    if i%50 == 0:
        print('step',i)
print('done')

step 2000
Didn't work Room: 12535946
step 2050
Didn't work Room: 12774249
Didn't work Room: 12798141
step 2100
step 2150
step 2200
step 2250
step 2300
step 2350
step 2400
step 2450
Didn't work Room: 15062944
done


In [82]:
#Generating next entries
for i in range(2500,3000):
    rating_ = rating_.append(ratings(ids.iloc[i]))
    if i%50 == 0:
        print('step',i)
print('done')

step 2500
step 2550
step 2600
step 2650
step 2700
Didn't work Room: 16804336
step 2750
step 2800
step 2850
step 2900
Didn't work Room: 18209795
Didn't work Room: 18264659
step 2950
done


In [83]:
#Generating next entries
for i in range(3000,3500):
    rating_ = rating_.append(ratings(ids.iloc[i]))
    if i%50 == 0:
        print('step',i)
print('done')

step 3000
step 3050
step 3100
Didn't work Room: 19495570
step 3150
step 3200
step 3250
step 3300
step 3350
Didn't work Room: 21133911
step 3400
Didn't work Room: 21324522
step 3450
done


In [84]:
bak3500_=rating_ #Backup dataframe

In [85]:
#Generating next entries
for i in range(3500,4000):
    rating_ = rating_.append(ratings(ids.iloc[i]))
    if i%50 == 0:
        print('step',i)
print('done')

step 3500
step 3550
Didn't work Room: 21892834
step 3600
step 3650
Didn't work Room: 22265878
step 3700
step 3750
Didn't work Room: 22881110
step 3800
Didn't work Room: 23336678
Didn't work Room: 23378803
step 3850
step 3900
step 3950
done


In [86]:
#Generating next entries
for i in range(4000,4500):
    rating_ = rating_.append(ratings(ids.iloc[i]))
    if i%50 == 0:
        print('step',i)
print('done')

step 4000
step 4050
step 4100
step 4150
step 4200
step 4250
Didn't work Room: 26313558
Didn't work Room: 26318555
step 4300
step 4350
Didn't work Room: 27230145
step 4400
step 4450
Didn't work Room: 27926975
done


In [87]:
#Generating next entries
for i in range(4500,5000):
    rating_ = rating_.append(ratings(ids.iloc[i]))
    if i%50 == 0:
        print('step',i)
print('done')

step 4500
Didn't work Room: 28438004
step 4550
step 4600
step 4650
step 4700
step 4750
step 4800
step 4850
Didn't work Room: 30232726
step 4900
Didn't work Room: 30388429
step 4950
done


In [88]:
#Generating next entries
for i in range(5000,len(ids)):
    rating_ = rating_.append(ratings(ids.iloc[i]))
    if i%50 == 0:
        print('step',i)
print('done')

step 5000
step 5050
step 5100
step 5150
step 5200
Didn't work Room: 31802641
step 5250
done


In [89]:
bakfull_=rating_ #Backup dataframe

Generating a new dataframe to replace failed attempts of several listings

In [101]:
missing_=[3818298,4121110,6471117,7034609,7279583,9022118,9249597,9296089,9405466,9839017,10367220,
          10704981,12535946,12774249,12798141,15062944,16804336,18209795,18264659,19495570,21133911,
          21324522,21892834,22265878,22881110,23336678,23378803,26313558,26318555,27230145,27926975,
          28438004,30232726,30388429,31802641]

miss_df=pd.DataFrame(missing_, columns=['id'])
ids_miss = miss_df['id']
print(ids_miss.head())
print('DataPoints:',len(ids_miss))

0    3818298
1    4121110
2    6471117
3    7034609
4    7279583
Name: id, dtype: int64
DataPoints: 35


In [102]:
#Generating Data Frame for Missed Listings
rating_miss = ratings(ids_miss.iloc[0])
#Generating next entries
for i in range(1,len(ids_miss)):
    rating_miss = rating_miss.append(ratings(ids_miss.iloc[i]))
    if i%5 == 0:
        print('step',i)
print('done')

step 5
step 10
step 15
step 20
step 25
step 30
done


In [115]:
#Complementing missed listings
bak_=rating_.drop(missing_) #Dropping initially missed listings with succesful attempts

complete_=pd.concat([bak_,rating_miss])
complete_.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5296 entries, 3647 to 31802641
Data columns (total 7 columns):
Stars            4591 non-null object
Accuracy         4591 non-null object
Communication    4591 non-null object
Cleanliness      4591 non-null object
Location         4591 non-null object
Check-In         4591 non-null object
Value            4591 non-null object
dtypes: object(7)
memory usage: 331.0+ KB


In [123]:
#Generating csv file to export Ratings
complete_.to_csv('../files/ratings.csv')